In [1]:
%autoreload
from analytics.util.notebook_config import *
#set_notebook_style()
show_more_data()

from IPython.core.display import display, HTML
display(HTML('''<style>
.container {width:98% !important;}
.dataframe th{font: bold 14px times; background: #fa8072; text-align: right;}
.dataframe td{font: 14px courier; background: #afeeee; text-align: right;}
.output_subarea.output
_text.output_stream.output_stderr {background: #fff; font-style: italic;}
</style>'''))




import pandas
pandas.set_option("display.max_rows",999,"display.max_columns", 999)


ERROR:root:Line magic function `%autoreload` not found.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

DF  = pandas.read_csv('Twodraw.csv', dtype={'public_client_id':str})
DF  = DF.convert_objects(convert_numeric=True)
DF.shape

/opt/conda/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


(679, 1761)

In [3]:
############################################################################
############################################################################
########################        renew                #######################
############################################################################
############################################################################

df = DF.loc[(DF.RENEW==0)|(DF.RENEW==1)]
df.shape

(679, 1761)

In [4]:
df2 = df.drop(['public_client_id','RENEW','STATUS'], axis=1)
df3 = df2.replace(np.inf, np.nan)
df3 = df3.fillna(df.median())

print (df3.shape)

#############################################
########     standard deviation    #########
from sklearn import preprocessing
df4 = pandas.DataFrame(preprocessing.StandardScaler().fit(df3).transform(df3))
print (df4.shape)


(679, 1758)
(679, 1758)


In [ ]:
################################################################################
################################################################################
######################### random forest method #################################
################################################################################
################################################################################

from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import numpy as np
np.random.seed(10)

import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,GradientBoostingClassifier)
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline
from sklearn import  metrics


In [ ]:
y = df.RENEW
X = df4

n_folds = 3
B_P     = pandas.DataFrame()
AUC     = []
Import  = pandas.DataFrame()

N_estimate = [1000]
M_features = ['sqrt','log2', 0.1, 0.3]
N_samples_leaf = [1, 3, 5]


for i in range(0,100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    XX_train, XX_test, Xy_train, Xy_test = train_test_split(X_train, y_train, test_size=0.2)
    
    
    A_S = pandas.DataFrame()
    
    for ne in N_estimate:
        for mf in M_features:
            for ns in N_samples_leaf:
                rf = RandomForestClassifier(n_estimators=ne, n_jobs=4, max_features=mf, min_samples_leaf=ns)
                this_scores = cross_val_score(rf, XX_train, Xy_train, cv=n_folds, n_jobs=4)

                a_s = pandas.DataFrame({'N_estimate': [ne], 'cv_score': [np.mean(this_scores)],'Max_feature':[mf], 'Min_sample_leaf':[ns] })
                A_S = pandas.concat([A_S, a_s], axis=0)
    
    A_S  = A_S.reset_index()
    ne_p = A_S[A_S['cv_score'] == A_S['cv_score'].max()]['N_estimate'].values[0]
    mf_p = A_S[A_S['cv_score'] == A_S['cv_score'].max()]['Max_feature'].values[0]
    ns_p = A_S[A_S['cv_score'] == A_S['cv_score'].max()]['Min_sample_leaf'].values[0]
    
    B_P  = pandas.concat([B_P, A_S[A_S['cv_score'] == A_S['cv_score'].max()]], axis=0)
        
    rf = RandomForestClassifier(n_estimators=ne_p, max_features=mf_p, min_samples_leaf=ns_p, n_jobs=4)
    rf.fit(X_train, y_train)

    y_pre_rf = rf.predict_proba(X_test)[:,1]
    fpr, tpr, _ = roc_curve(y_test, y_pre_rf,pos_label=1)
    auc = metrics.auc(fpr, tpr)
    AUC.append(auc)
    
    importance = pandas.DataFrame(rf.feature_importances_, index = df3.columns)
    importance = importance[importance[0]>0]
    Import     = pandas.concat([Import,importance], axis=0)
    

In [10]:
print (B_P.shape)
B_P.head(3)

(103, 5)


,index,Max_feature,Min_sample_leaf,N_estimate,cv_score
10,0,0.3,3,1000,0.668316
10,0,0.3,3,1000,0.691252
10,0,0.3,3,1000,0.652080


In [11]:
pandas.Series(AUC).hist()

print (pandas.Series(AUC).max())
print (pandas.Series(AUC).mean())
print (pandas.Series(AUC).min())

0.835616438356
0.752751999401
0.653837790824


In [12]:
Importance = Import.reset_index().groupby('index').mean()
Importance['appear_time'] = Import.reset_index().groupby('index').size()
Importance.columns = ['Importance_random_forest', 'appear_time']
#Importance.sort_values('Importance_random_forest', ascending= False).head(50)

Mean = df.groupby('STATUS').mean().transpose()

Summary = Importance.join(Mean)
Summary['direction'] = (Summary['renew'] - Summary['non_renew']).apply(lambda x: "+" if x>0 else ("-" if x<0 else 'na'))
Summary.sort_values('Importance_random_forest', ascending= False)[['Importance_random_forest','direction','renew','non_renew']].head(50)

,Importance_random_forest,direction,renew,non_renew
index,,,,
journey_2nd,0.071400,+,0.718499,0.359477
race1,0.017931,+,0.903485,0.718954
journey_two,0.014356,+,1.233244,0.800654
journey_binary,0.013575,+,0.820375,0.578431
last_call_time_interval,0.010799,-,19.546917,29.673203
enterpriseSabey,0.010202,+,0.179625,0.022876
enterpriseDIRECT,0.007388,-,0.820375,0.973856
n_day_use,0.005220,+,244.362857,210.343284
client_age,0.004977,+,53.367046,50.969575
